In [16]:

from pathlib import Path
from ast import literal_eval


import os, json
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumRegister
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter, SuzukiTrotter
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as AerSampler
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import SamplerV2 as Sampler, QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from datetime import datetime
import wesszumino as wz
from qiskit.quantum_info import SparsePauliOp, PauliList
from scipy.sparse.linalg import eigsh
path = os.path.join( r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\open-apikey.json")
#path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\apikey.json"
with open(path, encoding="utf-8") as f:
    api_key = json.load(f).get("apikey")

IBM_QUANTUM_API_KEY = api_key
ibm_instance_crn = "crn:v1:bluemix:public:quantum-computing:us-east:a/3ff62345f67c45e48e47a7f57d2f39f5:83214c75-88ab-4e55-8a87-6502ecc7cc9b::" #Open
#ibm_instance_crn = "crn:v1:bluemix:public:quantum-computing:us-east:a/d4f95db0515b47b7ba61dba8a424f873:ed0704ac-ad7d-4366-9bcc-4217fb64abd1::" #NQCC

service = QiskitRuntimeService(channel="ibm_quantum_platform", token=IBM_QUANTUM_API_KEY, instance=ibm_instance_crn)
COMPILE_BACKEND_NAME = "ibm_torino"#"ibm_marrakesh"
compile_backend = service.backend(COMPILE_BACKEND_NAME)
compile_target = compile_backend.target

# aer_backend = AerSimulator(method="statevector")
# compile_target = aer_backend.target

qiskit_runtime_service._discover_account:WARNING:2026-02-22 13:03:45,687: Loading account with the given token. A saved account will not be used.


In [2]:
def sparsepauliop_remove_each_term(spo: SparsePauliOp, simplify=True, atol=1e-12):
    """
    Return a list of new SparsePauliOp objects, each with exactly one term removed.
    Also returns metadata about which term was removed.
    """
    labels = spo.paulis.to_labels()
    coeffs = np.asarray(spo.coeffs, dtype=complex)

    out = []
    for i in range(len(labels)):
        new_labels = labels[:i] + labels[i+1:]
        new_coeffs = np.concatenate([coeffs[:i], coeffs[i+1:]])

        new_spo = SparsePauliOp(new_labels, new_coeffs)

        if simplify:
            # combines duplicate Pauli strings, drops near-zero coeffs
            new_spo = new_spo.simplify(atol=atol)

        out.append(
            {
                "removed_index": i,
                "removed_label": labels[i],
                "removed_coeff": coeffs[i],
                "op": new_spo,
            }
        )
    return out

def _kq_only_depth(qc, k=2, ignore_ops=("barrier", "measure")):
    """Depth counting only k-qubit operations (ignoring selected ops)."""
    qc_k = QuantumCircuit(qc.num_qubits, qc.num_clbits)

    q_index = {q: i for i, q in enumerate(qc.qubits)}
    c_index = {c: i for i, c in enumerate(qc.clbits)}

    for inst in qc.data:
        op = inst.operation
        if op.name in ignore_ops:
            continue
        if op.num_qubits == k:
            qargs = [qc_k.qubits[q_index[q]] for q in inst.qubits]
            cargs = [qc_k.clbits[c_index[c]] for c in inst.clbits] if inst.clbits else []
            qc_k.append(op, qargs, cargs)

    return qc_k.depth()

def circuit_cost_metrics(qc):

    ops = qc.count_ops()
    ops_str = {str(k): int(v) for k, v in ops.items()}

    ignore_ops = {"barrier", "measure"}

    n2q = sum(1
        for inst in qc.data
        if inst.operation.num_qubits == 2 and inst.operation.name not in ignore_ops
    )

    n3q = sum(1
        for inst in qc.data
        if inst.operation.num_qubits == 3 and inst.operation.name not in ignore_ops
    )

    return {
        "depth": qc.depth(),
        "size": qc.size(),
        "num_2q_ops": n2q,
        "depth_2q": _kq_only_depth(qc, k=2),
        "num_3q_ops": n3q,
        "depth_3q": _kq_only_depth(qc, k=3),
        "count_ops": ops_str,
    }

def create_circuit(optimization_level, num_qubits, H_pauli, t_k, num_trotter_steps, seed=42):
    
    qr = QuantumRegister(num_qubits)
    qc = QuantumCircuit(qr)

    #evol_gate = PauliEvolutionGate(H_pauli, time=t_k, synthesis=SuzukiTrotter(order=2, reps=num_trotter_steps))
    evol_gate = PauliEvolutionGate(H_pauli,time=t_k,synthesis=LieTrotter(reps=num_trotter_steps))
    qc.append(evol_gate, qr)


    qc.measure_all()

    target = compile_target
    pm = generate_preset_pass_manager(target=target, optimization_level=optimization_level, seed_transpiler=seed)
    circuit_isa = pm.run(qc)
    
    return circuit_isa, circuit_cost_metrics(circuit_isa)

def get_cost(N, cutoff, seed):

    a = 1.0
    c = -0.2
    potential = "linear"
    boundary_condition = 'dirichlet'
    optimization_level = 0

    num_qubits = N*int(1+np.log2(cutoff))

    H_pauli, num_qubits = wz.build_wz_hamiltonian(cutoff,N,a, c=c,m=1.0,potential=potential,boundary_condition=boundary_condition)
    qc, initial_circuit_cost = create_circuit(optimization_level, num_qubits, H_pauli, t_k=1.0, num_trotter_steps=1, seed=seed)


    H = H_pauli.to_matrix(sparse=True)
    min_eigenvalue = eigsh(H, k=1, which="SA", return_eigenvectors=False)[0].real

    variants = sparsepauliop_remove_each_term(H_pauli)
    num_variants = len(variants)

    print(f"{num_variants} different H varaints")

    for i, v in enumerate(variants):
        print(f"variant [{i+1}/{num_variants}]")
        qc, circuit_cost = create_circuit(optimization_level, num_qubits, v['op'], t_k=1.0, num_trotter_steps=1, seed=seed)

        H_v = v['op'].to_matrix(sparse=True)
        v_e = eigsh(H_v, k=1, which="SA", return_eigenvectors=False)[0].real
        diff = np.abs(min_eigenvalue-v_e)

        variants[i]["diff"] = float(diff)
        variants[i]["cost"] = circuit_cost
        variants[i].pop('op')

    return initial_circuit_cost, variants


In [17]:
def _parse_complex_coeff(x):
    """
    Parse coefficients stored as strings like '(0.25+0j)' safely.
    Also handles numeric types directly.
    """
    if isinstance(x, (int, float, complex, np.number)):
        return complex(x)
    if isinstance(x, str):
        return complex(literal_eval(x))
    raise TypeError(f"Unsupported coeff type: {type(x)}")

def reconstruct_sparsepauliop_from_ablation_json(
    json_path,
    tol=1e-5,
    keep_if_equal=False,
    drop_identity=False,
):
    """
    Reconstruct original Hamiltonian from one-term-removal ablation JSON and prune by diff threshold.

    Parameters
    ----------
    json_path : str | Path
        Path to the saved JSON file.
    tol : float
        Terms with diff <= tol are removed (or < tol if keep_if_equal=True).
    keep_if_equal : bool
        If True, keep terms with diff == tol (i.e. remove only diff < tol).
    drop_identity : bool
        If True, also drop the all-identity term regardless of diff.

    Returns
    -------
    H_pruned : SparsePauliOp
        Reconstructed and pruned Hamiltonian.
    info : dict
        Summary metadata and lists of kept/removed terms.
    """
    json_path = Path(json_path)
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    variants = data["variants"]

    # Sort to reconstruct original order
    variants_sorted = sorted(variants, key=lambda v: v["removed_index"])

    labels = []
    coeffs = []
    diffs = []
    removed_indices = []

    n_qubits = None
    for v in variants_sorted:
        label = v["removed_label"]
        coeff = _parse_complex_coeff(v["removed_coeff"])
        diff = float(v["diff"])
        idx = int(v["removed_index"])

        if n_qubits is None:
            n_qubits = len(label)

        labels.append(label)
        coeffs.append(coeff)
        diffs.append(diff)
        removed_indices.append(idx)

    # Sanity check: indices should usually be contiguous [0, 1, ..., n-1]
    expected = list(range(len(variants_sorted)))
    if removed_indices != expected:
        print(
            "Warning: removed_index values are not contiguous/sorted as expected. "
            "Reconstruction still uses sorted removed_index order."
        )

    # Build keep mask based on diff threshold
    labels_kept = []
    coeffs_kept = []
    kept_terms = []
    dropped_terms = []

    for idx, label, coeff, diff in zip(removed_indices, labels, coeffs, diffs):
        is_identity = set(label) == {"I"}

        if drop_identity and is_identity:
            keep = False
            reason = "identity"
        else:
            if keep_if_equal:
                keep = diff >= tol  # drop only if diff < tol
            else:
                keep = diff > tol   # drop if diff <= tol

            reason = f"diff={diff:.3e}"

        term_record = {
            "index": idx,
            "label": label,
            "coeff": coeff,
            "diff": diff,
        }

        if keep:
            labels_kept.append(label)
            coeffs_kept.append(coeff)
            kept_terms.append(term_record)
        else:
            dropped_terms.append(term_record)

    # If everything got dropped, return an explicit zero operator of correct size
    if len(labels_kept) == 0:
        H_pruned = SparsePauliOp.from_list([("I" * n_qubits, 0.0)])
    else:
        H_pruned = SparsePauliOp.from_list(list(zip(labels_kept, coeffs_kept)))
        # Combine duplicates just in case (usually not needed if source was unique)
        H_pruned = H_pruned.simplify(atol=0.0, rtol=0.0)

    info = {
        "file": str(json_path),
        "potential": data.get("potential"),
        "cutoff": data.get("cutoff"),
        "N": data.get("N"),
        "basis": data.get("basis"),
        "n_qubits": n_qubits,
        "tol": tol,
        "keep_if_equal": keep_if_equal,
        "drop_identity": drop_identity,
        "n_terms_original": len(labels),
        "n_terms_kept": len(kept_terms),
        "n_terms_dropped": len(dropped_terms),
        "kept_terms": kept_terms,
        "dropped_terms": dropped_terms,
    }

    return H_pruned, info

In [6]:
def truncate_by_coeff_weight(pauli_coeffs, pauli_labels, keep_ratio=0.999, min_keep=0):

    c = np.asarray(pauli_coeffs)
    lab = np.asarray(pauli_labels)

    abs_c = np.abs(c)
    order = np.argsort(abs_c)[::-1]
    abs_sorted = abs_c[order]
    w = abs_sorted**2 #squared gives more aggressive truncation

    cum = np.cumsum(w)
    total = float(cum[-1])
    target = keep_ratio * total

    m = int(np.searchsorted(cum, target, side="left") + 1)
    m = max(m, int(min_keep))
    m = min(m, len(c))

    keep_idx = order[:m]
    truncated = float(total - cum[m-1])

    info = {
        "m": m,
        "n": len(pauli_coeffs),
        "keep_frac_terms": m / len(pauli_coeffs),
        "keep_ratio": keep_ratio,
        "truncated": truncated,
        "total_weight": total
    }
    return c[keep_idx], lab[keep_idx], keep_idx, info

In [18]:
N=3
cutoff=16
basis="Fock"
potential="linear"
boundary_condition="dirichlet"

keep_ratio=0.999

In [22]:
folder = 'N'+ str(N)

H_path = os.path.join(r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\Analyses\Model Checks\HamiltonianData", boundary_condition, potential, folder, f"{potential}_{cutoff}.json")
with open(H_path, 'r') as file:
    H_data = json.load(file)

full_pauli_coeffs = H_data["pauli_coeffs"]
full_pauli_labels = H_data["pauli_labels"]
pauli_terms = list(zip(full_pauli_coeffs, full_pauli_labels))

H_pauli, _ = wz.build_wz_hamiltonian(cutoff,N,a=1.0,c=0.0, m=1.0,potential=potential,boundary_condition=boundary_condition)

pauli_coeffs = np.real(H_pauli.coeffs).astype(float).tolist()
pauli_labels = H_pauli.paulis.to_labels()

kept_coeffs, kept_labels, keep_idx, trunc_info = truncate_by_coeff_weight(pauli_coeffs, pauli_labels, keep_ratio=keep_ratio)
keep_idx = np.sort(keep_idx)
kept_coeffs = np.asarray(pauli_coeffs)[keep_idx]
kept_labels = np.asarray(pauli_labels)[keep_idx]

H_pauli = SparsePauliOp(PauliList(kept_labels.tolist()), kept_coeffs.tolist())

num_qubits = N*int(1+np.log2(cutoff))
optimization_level=0
seed=42
qc, circuit_cost = create_circuit(optimization_level, num_qubits, H_pauli, t_k=1.0, num_trotter_steps=1, seed=seed)
circuit_cost

{'depth': 2594,
 'size': 6750,
 'num_2q_ops': 1052,
 'depth_2q': 597,
 'num_3q_ops': 0,
 'depth_3q': 0,
 'count_ops': {'rz': 3077,
  'sx': 2606,
  'cz': 1052,
  'measure': 15,
  'barrier': 1}}

In [20]:
len(H_pauli)

128

In [26]:
ml_path = os.path.join(r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\Qiskit\SKQD\ML-Testing\ML-Testing", potential, folder, f"L{cutoff}_fock.json")
tol = 1e-3

H_pruned, info = reconstruct_sparsepauliop_from_ablation_json(
    ml_path,
    tol=tol,
    keep_if_equal=False,   # removes diff <= tol
    drop_identity=False,   # set True if you want to always drop the constant shift term
)

qc, pruned_circuit_cost = create_circuit(optimization_level, num_qubits, H_pruned, t_k=1.0, num_trotter_steps=1, seed=seed)


print(f"File: {info['file']}")
print(f"Potential={info['potential']}, cutoff={info['cutoff']}, N={info['N']}, basis={info['basis']}")
print(f"n_qubits = {info['n_qubits']}")
print(f"Circuit cost: {pruned_circuit_cost}")
print(f"Original terms = {info['n_terms_original']}")
print(f"Kept terms     = {info['n_terms_kept']}")
print(f"Dropped terms  = {info['n_terms_dropped']} (tol={info['tol']})")

print("\nDropped terms (first 10):")
for t in info["dropped_terms"][:10]:
    print(f"  idx={t['index']:>3d}  {t['label']}  coeff={t['coeff']}  diff={t['diff']:.3e}")

print("\nReconstructed pruned SparsePauliOp:")
print(H_pruned)

File: C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\Qiskit\SKQD\ML-Testing\ML-Testing\linear\N3\L16_fock.json
Potential=linear, cutoff=16, N=3, basis=Fock
n_qubits = 15
Circuit cost: {'depth': 4901, 'size': 10820, 'num_2q_ops': 1795, 'depth_2q': 1208, 'num_3q_ops': 0, 'depth_3q': 0, 'count_ops': {'rz': 4672, 'sx': 4338, 'cz': 1795, 'measure': 15, 'barrier': 1}}
Original terms = 1146
Kept terms     = 167
Dropped terms  = 979 (tol=0.001)

Dropped terms (first 10):
  idx= 53  IIIIIIIXXZIIIII  coeff=(-0.031389141305977586+0j)  diff=2.014e-04
  idx= 55  IIIIIIIYYZIIIII  coeff=(-0.031389141305977586+0j)  diff=1.240e-04
  idx= 57  IIIIIIIZXZIIIII  coeff=(-0.000505896488950941+0j)  diff=7.200e-05
  idx= 59  IIIIIIXXXZIIIII  coeff=(-0.015655728135791983+0j)  diff=1.440e-05
  idx= 61  IIIIIIXYYZIIIII  coeff=(0.015655728135791983+0j)  diff=1.393e-05
  idx= 63  IIIIIIYXYZIIIII  coeff=(-0.015655728135791983+0j)  diff=1.394e-05
  idx= 65  IIIIIIYYXZIIIII  coe